# Bayesian Networks and HMM

$
\newcommand{\x}{\mathbf{x}}
\newcommand{\y}{\mathbf{y}}
$

| | |
|--|--|
| **Names** | *Thomas Brus & Jan Ubbo van Baardewijk* |
| **Group** | *ML_HMI_01* |


In this lab, you are given a basic implementation of the components of Factor graphs and the sum-product algorithm. With this, you can compute marginal probabilities and, by incorporating extra factors, joint probabilities of nodes and observations. Familiarise yourself with the code, and then answer the questions below.


In [2]:
%matplotlib inline
import numpy as np
import scipy.stats as st
import scipy.ndimage as ndim
import matplotlib.pyplot as plt
import operator
import pylab

pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [3]:
class Factor:
    """Implement a factor node in a Bayesian Network"""
    def __init__(self,name,factor):
        """Initialisation with the name of the node, and the implemented function of the connected variables
        
        The function `factor` must take a tuple as argument, with as many elements in the tuple as there 
        are connected variables to the factor node and the elements of the argument being the indices of the 
        possible values in the variable node. It returns the factor value for that setting of variables.
        
        Example: a factor implementing p(A), with A being a boolean variable with possible values {0: True, 1: False},
        factor((1,)) should return p(~a).
        """
        self.name = name
        self.fn = factor
        self.connections = [] # Pointer to the nodes this factor is connected to
        self.msgRcvd = []     # Stores messages received from the connected nodes
        self.msgSent = []     # Has a message been sent to the connected nodes
        self.nMsgsRcvd = 0    # number of messages received
    
    def addConnection(self, node):
        """Register a connection to a variable. The connections must be added to the factor node
        in the same order as the indices are expected by the factor function
        
        This automatically registers the inverse connection from variable to the factor."""
        self.connections.append(node)
        self.msgSent.append(False)
        self.msgRcvd.append(None)
        node.addConnection(self)
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return "Factor "+self.name
        
    def recPrint(self, parent=None, indent=""):
        """Recursively print the tree"""
        print indent,self.__str__()
        for i in range(len(self.connections)):
            if self.connections[i]!=parent:
                self.connections[i].recPrint(self,indent+"  ")
            
    def recvMsg(self, sender, msg):
        """Receive a message `msg` from a connected `sender` variable node"""
#         print self.name,"Receiving message from",sender,":",msg
        for i in range(len(self.connections)):
            if self.connections[i] == sender:
                if self.msgRcvd[i]!=None:
                    print "... ignoring"
                    return
                self.msgRcvd[i] = msg
                self.nMsgsRcvd += 1
                return
        print "ERROR:",self.__str__(),"could not find the corresponding variable",sender.__str__()
   
    def resetMsgs(self):
        self.nMsgsRcvd=0
        self.msgRcvd = [ None for m in self.msgRcvd ]
        self.msgSent = [ False for m in self.msgSent ]

    def recursiveEnumerate(self, outIndex, valIndices, values, res):
        """Stores, in list `res`, for each possible value of the node connected to connection `outindex`,
            the sum over all possible combinations of values of the incoming connections of the product 
            of the factor of these values with the messages with these values. (Aaaargh.) """
        if len(valIndices) == len(self.connections): # end of recursion
            prod = self.fn(tuple(valIndices))
            for i in range(len(valIndices)):
                if i != outIndex:
                    prod *= self.msgRcvd[i][valIndices[i]]
            
            res[valIndices[outIndex]] += prod 
            return
        
        for i,v in enumerate(self.connections[len(valIndices)].values):
            self.recursiveEnumerate(outIndex,valIndices+[i],values+[v],res)
        
        
    def sendMessages(self):
        """Send a message to all connections on which no message has been sent yet, if all
        other connections have received a message already"""        
        
        # Special case: single connection
        if not self.msgSent[0] and len(self.connections) == 1:
            self.msgSent[0] = True    # Record that a message has been sent on that connection
            self.connections[0].recvMsg(self,np.array([self.fn((i,)) for i,v in enumerate(self.connections[0].values) ]))
            return True
        
        if self.nMsgsRcvd+1<len(self.connections): # More than one connection without incoming message
            return False                           # Can't send anything
        
        # If one connection has not received it's message, send a msg out on that connection
        if self.nMsgsRcvd+1 == len(self.connections): 
            for i in range(len(self.connections)):    # For connections on which we could send a message                
                if self.msgRcvd[i] is None:           # Find the one that hasn't received a message yet
                    if not self.msgSent[i]:           # If no message has been sent on that connection yet
                        msg = np.array([ 0. for c in self.connections[i].values ]) # Create a message with an entry for each possible value of the outgoing node
                        self.recursiveEnumerate(i,[],[],msg) # Compute the message value
                        self.connections[i].recvMsg(self,msg) 
                        self.msgSent[i] = True
                        return True
            
        if self.nMsgsRcvd == len(self.connections): # message received on all connections, send wherever not done yet.
            sentAny = False
            for i in range(len(self.connections)):       # For connections on which we could send a message
                if self.msgSent[i]:                      # ignore if handled already
                    continue
                msg = np.array([ 0. for c in self.connections[i].values ])
                self.recursiveEnumerate(i,[],[],msg)
                self.connections[i].recvMsg(self,msg)
                self.msgSent[i] = True
                sentAny = True
            return sentAny
                            
        return False
            
    

In [4]:
class Variable:
    """Implement a Variable node with categorical values"""
    def __init__(self,name,values):
        """Initialise with the name of the variable and a list of possible values for that variable"""
        self.name = name
        self.connections = []
        self.values = values
        self.msgRcvd = []     # Stores messages received from the connected nodes
        self.msgSent = []     # Has a message been sent to the connected nodes
        self.nMsgsRcvd = 0    # number of messages received
    
    def addConnection(self, node):
        """Add a connection to a factor node. This should never be called explicitly, except by
        the addConnection method of the Factor class"""
        self.connections.append(node)
        self.msgSent.append(False)
        self.msgRcvd.append(None)
        
    def __str__(self):
        return self.name
    def __repr__(self):
        return "Var "+self.name
    
    
    def recPrint(self, parent=None, indent=""):
        """Recursively print the tree"""
        print indent,self.__str__()
        for i in range(len(self.connections)):
            if self.connections[i]!=parent:
                self.connections[i].recPrint(self,indent+"  ")        
    
    def recvMsg(self, sender, msg):
        """Receive a message `msg` from Factor node `sender`"""
#         print self.name,"Receiving message from",sender,":",msg
        for i in range(len(self.connections)):
            if self.connections[i] == sender:
                if self.msgRcvd[i]!=None:
                    print "WARNING: ignoring a message"
                    return
                self.msgRcvd[i] = msg
                self.nMsgsRcvd += 1
                return
        print "ERROR:",self.__str__(),"could not find the corresponding factor",sender.__str__()
        
    def resetMsgs(self):
        self.nMsgsRcvd=0
        self.msgRcvd = [ None for m in self.msgRcvd ]
        self.msgSent = [ False for m in self.msgSent ]
        
    
    def sendMessages(self):
        """Send a message to all connections on which no message has been sent yet, if all
        other connections have received a message already
        """    
        # Special case: single connection
        if not self.msgSent[0] and len(self.connections) == 1:
            self.msgSent[0] = True
            self.connections[0].recvMsg(self,np.array([1. for v in self.values]))
            return True
        
        # If one connection has not received it's message, send a msg out on that connection
        if self.nMsgsRcvd+1 == len(self.connections): 
            con = 0
            for i in range(len(self.connections)):    # For connections on which we could send a message                
                if self.msgRcvd[i] == None:           # Find the one that hasn't received a message yet
                    con = i
                    break
                    
            if self.msgSent[con]:                     # Don't send it again
                return False
            
            msg = np.ones(len(self.values))           # The message is the product of the 
            for i in range(len(self.connections)):    # incoming messages on all connections,
                if i == con:                          # except the outgoing one
                    continue
                msg *= self.msgRcvd[i]
            self.connections[con].recvMsg(self,msg)   # Send the message
            self.msgSent[con] = True
            return True
            
        if self.nMsgsRcvd == len(self.connections): # message received on all connections, send wherever not done yet.
            sentAny = False
            for con in range(len(self.connections)):       # For connections on which we could send a message
                if self.msgSent[con]:                      # ignore if handled already
                    continue
                msg = np.ones(len(self.values))            # Initialise the message
                for i in range(len(self.connections)):     # multiply with all relevant incoming messages
                    if i == con:
                        continue
                    msg *= self.msgRcvd[i]
                self.connections[con].recvMsg(self,msg)    # and send it out
                self.msgSent[con] = True                
                sentAny = True
            return sentAny

        return False
    
    def getMarginals(self):
        """Once all messages have been received, the marginal probability 
        (or joint probability with specific values of other nodes, if such 
        factors have been implemented), is the product of all incoming messages"""
        if self.nMsgsRcvd != len(self.connections):        # Check that all messages have been received
            print "# message = ", self.nMsgsRcvd
            print "# connections = ", len(self.connections)
            print "ERROR printing marginals: messages missing"
        prod = self.msgRcvd[0]                             # Compute the product
        for m in self.msgRcvd[1:]:
            prod *= m
        return prod                              

In [5]:
class Graph:
    def __init__(self):
        self.var = {}
        self.fact = {}
    def addVariables(self,values, *names):
        for n in names:
            self.var[n] = Variable(n,values)
    def addFactor(self,name,fTable, *connections):
        self.fact[name] = Factor(name,lambda(v) : fTable[v])
        for c in connections:
            self.fact[name].addConnection(self.var[c])
    def propagateMessages(self):
        hasSent = True
        while hasSent:
            hasSent = False
            for k,f in self.fact.iteritems():
                hasSent |= f.sendMessages()
            for k,v in self.var.iteritems():
                hasSent |= v.sendMessages()
    def printMarginals(self):
        # Print out the marginals
        for k,v in self.var.iteritems():
            print v,v.getMarginals()
            
    def getMarginals(self, n):
        return self.var[n].getMarginals()
    
    def printTree(self):
        for k,v in self.var.iteritems():
            print v.recPrint();

    def reset(self):
        for k,f in self.fact.iteritems():
            f.resetMsgs()
        for k,v in self.var.iteritems():
            v.resetMsgs()


pB = np.array([1./12,11./12])
pC = np.array([1./3,2./3])
pA_BC = np.array([[[0, 0.25],[.5,1.]],[[1,.75],[.5,0.]]])
pD_A = np.array([[0,.75],[1,.25]])

g = Graph()
g.addVariables(["True","False"], "A","B","C","D")
g.addFactor("p(B)", pB, "B")
g.addFactor("p(C)", pC, "C")
g.addFactor("p(A|B,C)", pA_BC, "A","B","C")
g.addFactor("P(D|A)",pD_A,"D","A")

g.propagateMessages()
g.printMarginals()
g.reset()

# g.addFactor("obs B", np.array([1.,0.]),"B")
# g.propagateMessages()
# g.printMarginals()

# g.printTree();

A [ 0.77777778  0.22222222]
C [ 0.33333333  0.66666667]
B [ 0.08333333  0.91666667]
D [ 0.16666667  0.83333333]


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


** Question 1 [10 marks]** Using the above code, implement the computation of the following probabilities:

* $p(B|\neg A)$
* $p(C|\neg A)$
* $p(C|\neg A,\neg B)$
* $p(\neg A, \neg B, C, \neg D)$
* $p(B,C|D)$

In [6]:
# Answer Q1

# https://www.cl.cam.ac.uk/teaching/1011/L101/ml4lp-lect3.pdf

g.addFactor("obs -A", np.array([0.,1.]) ,"A")
g.propagateMessages()

# Answer Q1.1
print "Q1.1:"
print "p(B|-A) = ", g.getMarginals("B")[0]
# print "p(-B|-A) = ", g.getMarginals("B")[1]

# Answer Q1.2
print "\nQ1,2:"
print "p(C|-A) = ", g.getMarginals("C")[0]

# print "p(D|-A) = ", g.getMarginals("D")[0]

g.reset()

g.addFactor("obs -B", np.array([0.,1.]), "B")
g.propagateMessages()

# Answer Q1.3
print "\nQ1.3:"
print "p(C|-A,-B) = ", g.getMarginals("C")[0]

g.reset()

# Answer Q1.4
print "\nQ1.4:"
print "P(C|-A,-B) = (see Q1.3) = 0.152777777778"
print "P(-B|-A) = (see Q1.1) = 0.025462962963"
print "P(-A) = (see marginal probabilities of initial graph) = 0.22222222"
print "P(-D|-A) = (given) = 0.25"

print "P(-A,-B,C,-D) = P(C|-A,-B) * P(B|-A) * P(-A) * P(-D|-A) = ", 0.152777777778 * 0.025462962963 * 0.22222222 * 0.25

# Answer Q1.5
g2 = Graph()
g2.addVariables(["True","False"], "A","B","C","D")
g2.addFactor("p(B)", pB, "B")
g2.addFactor("p(C)", pC, "C")
g2.addFactor("p(A|B,C)", pA_BC, "A","B","C")
g2.addFactor("P(D|A)",pD_A,"D","A")

g2.addFactor("obs A", np.array([1.,0.]), "A")
g2.propagateMessages()

print "\nQ1.5:"
print "p(B|A) = ", g2.getMarginals("B")[0]
print "p(C|A) = ", g2.getMarginals("C")[0]
print "p(D|A) = ", g2.getMarginals("D")[0]
print "P(A) = 0.77777778"

print "\nP(B|-A) = 0.0694444444444"
print "P(C|-A) = 0.180555555556"
print "P(D|-A) = 0.166666666667"
print "P(-A) = 0.22222222"

print "\nP(B,C,D) = P(B|A) * P(C|A) * P(D|A) * P(A) + P(B|-A) * P(C|-A) * P(D|-A) * P(-A) = ", 0.0138888888889 * 0.152777777778 * 0.0 * 0.77777778 + 0.0694444444444 * 0.180555555556 * 0.166666666667 * 0.22222222
print "P(D) = (see marginial probabilities of initial graph) = 0.16666667"

print "\nP(B,C|D) = P(B,C,D) / P(D) = ", 0.000464391856355 / 0.16666667

Q1.1:
p(B|-A) =  0.0694444444444

Q1,2:
p(C|-A) =  0.180555555556

Q1.3:
p(C|-A,-B) =  0.152777777778

Q1.4:
P(C|-A,-B) = (see Q1.3) = 0.152777777778
P(-B|-A) = (see Q1.1) = 0.025462962963
P(-A) = (see marginal probabilities of initial graph) = 0.22222222
P(-D|-A) = (given) = 0.25
P(-A,-B,C,-D) = P(C|-A,-B) * P(B|-A) * P(-A) * P(-D|-A) =  0.000216120825457

Q1.5:
p(B|A) =  0.0138888888889
p(C|A) =  0.152777777778
p(D|A) =  0.0
P(A) = 0.77777778

P(B|-A) = 0.0694444444444
P(C|-A) = 0.180555555556
P(D|-A) = 0.166666666667
P(-A) = 0.22222222

P(B,C,D) = P(B|A) * P(C|A) * P(D|A) * P(A) + P(B|-A) * P(C|-A) * P(D|-A) * P(-A) =  0.000464391856355
P(D) = (see marginial probabilities of initial graph) = 0.16666667

P(B,C|D) = P(B,C,D) / P(D) =  0.0027863510824


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.



Somebody is cheating at playing heads and tails: he's using a biased coin (probability of heads: 0.7). However, to avoid suspicion, he also uses a fair coin (probability of heads: 0.5) sometimes, and switches between the two coins occasionally. If he has just thrown a fair coin, he will switch to the biased coin with probability 0.2 (and keep the fair coin with probability 0.8). If he's using the biased coin, he'll switch to fair with probability 0.5. When he starts playing, he chooses a coin at random (p=0.5)

You're playing with this person and observe the following sequence of outcomes:

$$
HHTHHHTTTHTH
$$

**Question 2 [20 marks]** Using the above code, implement a graph corresponding to the above problem. This is an example of a Hidden Markov Model. Given the probabilities listed above, what is the probability of this sequence? For each throw, what is the probability that a fair coin was used? 


In [27]:
# Answer Q2

g = Graph()

g.addVariables(["Fair", "Unfair"], "S0")
g.addVariables(["Fair", "Unfair"], "S1")
g.addVariables(["Fair", "Unfair"], "S2")
g.addVariables(["Fair", "Unfair"], "S3")
g.addVariables(["Fair", "Unfair"], "S4")
g.addVariables(["Fair", "Unfair"], "S5")
g.addVariables(["Fair", "Unfair"], "S6")
g.addVariables(["Fair", "Unfair"], "S7")
g.addVariables(["Fair", "Unfair"], "S8")
g.addVariables(["Fair", "Unfair"], "S9")
g.addVariables(["Fair", "Unfair"], "S10")
g.addVariables(["Fair", "Unfair"], "S11")
g.addVariables(["Fair", "Unfair"], "S12")

g.addVariables(["Heads", "Tails"], "C1")
g.addVariables(["Heads", "Tails"], "C2")
g.addVariables(["Heads", "Tails"], "C3")
g.addVariables(["Heads", "Tails"], "C4")
g.addVariables(["Heads", "Tails"], "C5")
g.addVariables(["Heads", "Tails"], "C6")
g.addVariables(["Heads", "Tails"], "C7")
g.addVariables(["Heads", "Tails"], "C8")
g.addVariables(["Heads", "Tails"], "C9")
g.addVariables(["Heads", "Tails"], "C10")
g.addVariables(["Heads", "Tails"], "C11")
g.addVariables(["Heads", "Tails"], "C12")

g.addFactor("p(S1|S0)", np.array([ [0.5, 0.5], [0.5, 0.5] ]), "S0", "S1")

g.addFactor("p(C1|S1)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C1", "S1")
g.addFactor("p(S2|S1)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S1", "S2")

g.addFactor("p(C2|S2)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C2", "S2")
g.addFactor("p(S3|S2)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S2", "S3")

g.addFactor("p(C3|S3)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C3", "S3")
g.addFactor("p(S4|S3)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S3", "S4")

g.addFactor("p(C4|S4)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C4", "S4")
g.addFactor("p(S5|S4)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S4", "S5")

g.addFactor("p(C5|S5)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C5", "S5")
g.addFactor("p(S6|S5)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S5", "S6")

g.addFactor("p(C6|S6)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C6", "S6")
g.addFactor("p(S7|S6)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S6", "S7")

g.addFactor("p(C7|S7)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C7", "S7")
g.addFactor("p(S8|S7)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S7", "S8")

g.addFactor("p(C8|S8)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C8", "S8")
g.addFactor("p(S9|S8)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S8", "S9")

g.addFactor("p(C9|S9)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C9", "S9")
g.addFactor("p(S10|S9)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S9", "S10")

g.addFactor("p(C10|S10)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C10", "S10")
g.addFactor("p(S11|S10)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S10", "S11")

g.addFactor("p(C11|S11)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C11", "S11")
g.addFactor("p(S12|S11)", np.array([ [0.8, 0.5], [0.2, 0.5] ]), "S11", "S12")

g.addFactor("p(C12|S12)", np.array([ [0.5, 0.7], [0.5, 0.3] ]), "C12", "S12")

# HHTHHHTTTHT(H)
g.addFactor("obs C1", np.array([1.0, 0.0]), "C1") # H
g.addFactor("obs C2", np.array([1.0, 0.0]), "C2") # H
g.addFactor("obs C3", np.array([0.0, 1.0]), "C3") # T
g.addFactor("obs C4", np.array([1.0, 0.0]), "C4") # H
g.addFactor("obs C5", np.array([1.0, 0.0]), "C5") # H
g.addFactor("obs C6", np.array([1.0, 0.0]), "C6") # H
g.addFactor("obs C7", np.array([0.0, 1.0]), "C7") # T
g.addFactor("obs C8", np.array([0.0, 1.0]), "C8") # T
g.addFactor("obs C9", np.array([0.0, 1.0]), "C9") # T
g.addFactor("obs C10", np.array([1.0, 0.0]), "C10") # H
g.addFactor("obs C11", np.array([0.0, 1.0]), "C11") # T

g.propagateMessages()

# 1) "Given the probabilities listed above, what is the probability of this sequence?"
print "P(HHTHHHTTTHTH) = ", g.getMarginals("C12")[0]

# 2) For each throw, what is the probability that a fair coin was used?
# FIXME: this outputs the the probability that a fair coin was used *given* the sequence???
r0 = g.getMarginals("S0")
r1 = g.getMarginals("S1")
r2 = g.getMarginals("S2")
r3 = g.getMarginals("S3")
r4 = g.getMarginals("S4")
r5 = g.getMarginals("S5")
r6 = g.getMarginals("S6")
r7 = g.getMarginals("S7")
r8 = g.getMarginals("S8")
r9 = g.getMarginals("S9")
r10 = g.getMarginals("S10")
r11 = g.getMarginals("S11")
print r0[0]/(r0[0]+r0[1])
print r1[0]/(r1[0]+r1[1])
print r2[0]/(r2[0]+r2[1])
print r3[0]/(r3[0]+r3[1])
print r4[0]/(r4[0]+r4[1])
print r5[0]/(r5[0]+r5[1])
print r6[0]/(r6[0]+r6[1])
print r7[0]/(r7[0]+r7[1])
print r8[0]/(r8[0]+r8[1])
print r9[0]/(r9[0]+r9[1])
print r10[0]/(r10[0]+r10[1])
print r11[0]/(r11[0]+r11[1])
# ...?

P(HHTHHHTTTHTH) =  0.000585467015581
0.5
0.624235079191
0.640496815789
0.75812523322
0.637186671992
0.612950498064
0.652524761107
0.809617389536
0.838613800196
0.81844866655
0.694228013676
0.746748167898


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:65: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
